<a href="https://colab.research.google.com/github/PratikStar/google-colab/blob/main/2_Creating_%26_Training_Audio_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


### Install tensorflow v2.4.0

In [2]:
!pip uninstall tensorflow
!pip install tensorflow==2.4.0

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.6.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 394.7 MB 18 kB/s 
     |████████████████████████████████| 462 kB 43.4 MB/s 
     |████████████████████████████████| 3.8 MB 37.1 MB/s 
     |████████████████████████████████| 2.9 MB 30.3 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    F

## VAE Code

In [3]:
import os
import pickle
import csv
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Flatten, Dense, Reshape, Conv2DTranspose, Activation, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import numpy as np
import tensorflow as tf


tf.compat.v1.disable_eager_execution()


class VAE:
    """
    VAE represents a Deep Convolutional variational autoencoder architecture
    with mirrored encoder and decoder components.
    """

    def __init__(self,
                 input_shape,
                 conv_filters,
                 conv_kernels,
                 conv_strides,
                 latent_space_dim):
        self.input_shape = input_shape
        self.conv_filters = conv_filters
        self.conv_kernels = conv_kernels 
        self.conv_strides = conv_strides 
        self.latent_space_dim = latent_space_dim 
        self.reconstruction_loss_weight = 1000000

        self.encoder = None
        self.decoder = None
        self.model = None

        self._num_conv_layers = len(conv_filters)
        self._shape_before_bottleneck = None
        self._model_input = None

        self._build()

    def summary(self):
        self.encoder.summary()
        self.decoder.summary()
        self.model.summary()

    def compile(self, learning_rate=0.0001):
        optimizer = Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss=self._calculate_combined_loss,
                           metrics=[self._calculate_reconstruction_loss,
                                    self._calculate_kl_loss])

    def train(self, x_train, batch_size, num_epochs):
        self.model.fit(x_train,
                       x_train,
                       batch_size=batch_size,
                       epochs=num_epochs,
                       shuffle=True)

    def save(self, save_folder="."):
        self._create_folder_if_it_doesnt_exist(save_folder)
        self._save_parameters(save_folder)
        self._save_weights(save_folder)

    def reconstruct(self, images):
        latent_representations = self.encoder.predict(images)
        reconstructed_images = self.decoder.predict(latent_representations)
        return reconstructed_images, latent_representations

    def _save_parameters(self, save_folder):
        parameters = [
            self.input_shape,
            self.conv_filters,
            self.conv_kernels,
            self.conv_strides,
            self.latent_space_dim
        ]
        save_path = os.path.join(save_folder, "parameters.pkl")
        with open(save_path, "wb") as f:
            print(parameters)
            pickle.dump(parameters, f)

    def _save_weights(self, save_folder):
        save_path = os.path.join(save_folder, "weights.h5")
        self.model.save_weights(save_path)

    @classmethod
    def load(cls, save_folder="."):
        parameters_path = os.path.join(save_folder, "parameters.pkl")
        with open(parameters_path, "rb") as f:
            parameters = pickle.load(f)
        autoencoder = VAE(*parameters)
        weights_path = os.path.join(save_folder, "weights.h5")
        autoencoder.load_weights(weights_path)
        return autoencoder

    def load_weights(self, weights_path):
        self.model.load_weights(weights_path)

    def _calculate_combined_loss(self, y_target, y_predicted):
        reconstruction_loss = self._calculate_reconstruction_loss(y_target, y_predicted)
        kl_loss = self._calculate_kl_loss(y_target, y_predicted)
        combined_loss = self.reconstruction_loss_weight * reconstruction_loss\
                                                         + kl_loss
        return combined_loss

    def _calculate_reconstruction_loss(self, y_target, y_predicted):
        error = y_target - y_predicted
        reconstruction_loss = K.mean(K.square(error), axis=[1, 2, 3])
        return reconstruction_loss

    def _calculate_kl_loss(self, y_target, y_predicted):
        kl_loss = -0.5 * K.sum(1 + self.log_variance - K.square(self.mu) -
                               K.exp(self.log_variance), axis=1)
        return kl_loss

    def _create_folder_if_it_doesnt_exist(self, folder):
        if not os.path.exists(folder):
            os.makedirs(folder)

    def _build(self):
        self._build_encoder()
        self._build_decoder()
        self._build_autoencoder()

    def _build_autoencoder(self):
        model_input = self._model_input
        model_output = self.decoder(self.encoder(model_input))
        self.model = Model(model_input, model_output, name="autoencoder")

    def _build_decoder(self):
        decoder_input = self._add_decoder_input()
        dense_layer = self._add_dense_layer(decoder_input)
        reshape_layer = self._add_reshape_layer(dense_layer)
        conv_transpose_layers = self._add_conv_transpose_layers(reshape_layer)
        decoder_output = self._add_decoder_output(conv_transpose_layers)
        self.decoder = Model(decoder_input, decoder_output, name="decoder")

    def _add_decoder_input(self):
        return Input(shape=self.latent_space_dim, name="decoder_input")

    def _add_dense_layer(self, decoder_input):
        num_neurons = np.prod(self._shape_before_bottleneck) # [1, 2, 4] -> 8
        dense_layer = Dense(num_neurons, name="decoder_dense")(decoder_input)
        return dense_layer

    def _add_reshape_layer(self, dense_layer):
        return Reshape(self._shape_before_bottleneck)(dense_layer)

    def _add_conv_transpose_layers(self, x):
        """Add conv transpose blocks."""
        # loop through all the conv layers in reverse order and stop at the
        # first layer
        for layer_index in reversed(range(1, self._num_conv_layers)):
            x = self._add_conv_transpose_layer(layer_index, x)
        return x

    def _add_conv_transpose_layer(self, layer_index, x):
        layer_num = self._num_conv_layers - layer_index
        conv_transpose_layer = Conv2DTranspose(
            filters=self.conv_filters[layer_index],
            kernel_size=self.conv_kernels[layer_index],
            strides=self.conv_strides[layer_index],
            padding="same",
            name=f"decoder_conv_transpose_layer_{layer_num}"
        )
        x = conv_transpose_layer(x)
        x = ReLU(name=f"decoder_relu_{layer_num}")(x)
        x = BatchNormalization(name=f"decoder_bn_{layer_num}")(x)
        return x

    def _add_decoder_output(self, x):
        conv_transpose_layer = Conv2DTranspose(
            filters=1,
            kernel_size=self.conv_kernels[0],
            strides=self.conv_strides[0],
            padding="same",
            name=f"decoder_conv_transpose_layer_{self._num_conv_layers}"
        )
        x = conv_transpose_layer(x)
        output_layer = Activation("sigmoid", name="sigmoid_layer")(x)
        return output_layer

    def _build_encoder(self):
        encoder_input = self._add_encoder_input()
        conv_layers = self._add_conv_layers(encoder_input)
        bottleneck = self._add_bottleneck(conv_layers)
        self._model_input = encoder_input
        self.encoder = Model(encoder_input, bottleneck, name="encoder")

    def _add_encoder_input(self):
        return Input(shape=self.input_shape, name="encoder_input")

    def _add_conv_layers(self, encoder_input):
        """Create all convolutional blocks in encoder."""
        x = encoder_input
        for layer_index in range(self._num_conv_layers):
            x = self._add_conv_layer(layer_index, x)
        return x

    def _add_conv_layer(self, layer_index, x):
        """Add a convolutional block to a graph of layers, consisting of
        conv 2d + ReLU + batch normalization.
        """
        layer_number = layer_index + 1
        conv_layer = Conv2D(
            filters=self.conv_filters[layer_index],
            kernel_size=self.conv_kernels[layer_index],
            strides=self.conv_strides[layer_index],
            padding="same",
            name=f"encoder_conv_layer_{layer_number}"
        )
        x = conv_layer(x)
        x = ReLU(name=f"encoder_relu_{layer_number}")(x)
        x = BatchNormalization(name=f"encoder_bn_{layer_number}")(x)
        return x

    def _add_bottleneck(self, x):
        """Flatten data and add bottleneck with Guassian sampling (Dense
        layer).
        """
        self._shape_before_bottleneck = K.int_shape(x)[1:]
        x = Flatten()(x)
        self.mu = Dense(self.latent_space_dim, name="mu")(x)
        self.log_variance = Dense(self.latent_space_dim,
                                  name="log_variance")(x)

        def sample_point_from_normal_distribution(args):
            mu, log_variance = args
            epsilon = K.random_normal(shape=K.shape(self.mu), mean=0.,
                                      stddev=1.)
            sampled_point = mu + K.exp(log_variance / 2) * epsilon
            return sampled_point

        x = Lambda(sample_point_from_normal_distribution,
                   name="encoder_output")([self.mu, self.log_variance])
        return x


if __name__ == "__main__":
    autoencoder = VAE(
        input_shape=(512, 64, 1),
        conv_filters=(32, 64, 64, 64),
        conv_kernels=(3, 3, 3, 3),
        conv_strides=(1, 2, 2, 1),
        latent_space_dim=32
    )
    autoencoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 512, 64, 1)] 0                                            
__________________________________________________________________________________________________
encoder_conv_layer_1 (Conv2D)   (None, 512, 64, 32)  320         encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_relu_1 (ReLU)           (None, 512, 64, 32)  0           encoder_conv_layer_1[0][0]       
__________________________________________________________________________________________________
encoder_bn_1 (BatchNormalizatio (None, 512, 64, 32)  128         encoder_relu_1[0][0]             
____________________________________________________________________________________________

2. Train VAE

## Train the VAE

In [4]:
import re
import os
import numpy as np

LEARNING_RATE = 0.0001
BATCH_SIZE = 50
EPOCHS = 100
LATENT_SPACE_DIM= 8

SPECTROGRAMS_PATH = "/content/drive/MyDrive/Music/VAE/spectrogram"

def load_fsdd(spectrograms_path):
    dataset = {}
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:

            if True: # re.match(regex, file_name):
                file_path = os.path.join(root, file_name)
                spectrogram = np.load(file_path) # (n_bins, n_frames, 1) 
                # print(spectrogram.shape)
                dataset[file_name] = spectrogram[..., np.newaxis]
    return dataset

def train(x_train, learning_rate, batch_size, epochs):
    autoencoder = VAE(
        input_shape=(256, 64, 1),
        conv_filters=(32, 16, 16, 8), # 16, 8, 8, 4 -> 
        conv_kernels=(3, 3, 3, 3),
        conv_strides=(1, 2, 2, 1),
        latent_space_dim=LATENT_SPACE_DIM
    )
    # autoencoder.summary()
    print(x_train.shape)
    autoencoder.compile(learning_rate)
    autoencoder.train(x_train, batch_size, epochs)
    return autoencoder


dataset = load_fsdd(SPECTROGRAMS_PATH)

autoencoder = train(np.array(list(dataset.values())), LEARNING_RATE, BATCH_SIZE, EPOCHS)
model_suffix = "-" + str(LATENT_SPACE_DIM) + "-" + str(BATCH_SIZE) + "-" + str(EPOCHS)
autoencoder.save("/content/drive/MyDrive/Music/VAE/model" + model_suffix)
print("Model Saved!!")

(960, 256, 64, 1)
Train on 960 samples
Epoch 1/100
960/960 [==============================] - 21s 22ms/sample - loss: 197509.0562 - _calculate_reconstruction_loss: 0.1972 - _calculate_kl_loss: 278.1357
Epoch 2/100
960/960 [==============================] - 3s 4ms/sample - loss: 175296.4017 - _calculate_reconstruction_loss: 0.1735 - _calculate_kl_loss: 1830.3674
Epoch 3/100
960/960 [==============================] - 3s 4ms/sample - loss: 167002.6115 - _calculate_reconstruction_loss: 0.1660 - _calculate_kl_loss: 965.8842
Epoch 4/100
960/960 [==============================] - 3s 4ms/sample - loss: 160650.7715 - _calculate_reconstruction_loss: 0.1597 - _calculate_kl_loss: 965.3743
Epoch 5/100
960/960 [==============================] - 3s 4ms/sample - loss: 155975.8009 - _calculate_reconstruction_loss: 0.1548 - _calculate_kl_loss: 1181.9280
Epoch 6/100
960/960 [==============================] - 3s 4ms/sample - loss: 144570.6790 - _calculate_reconstruction_loss: 0.1441 - _calculate_kl_loss: 

In [5]:
# # https://github.com/musikalkemist/generating-sound-with-neural-networks/blob/49d7db32c43d1a04c596cbbb282a9521be1e7fc8/11%20Implementing%20VAE/code/analysis.py

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display

LEARNING_RATE = 0.0001
BATCH_SIZE = 50
EPOCHS = 100
LATENT_SPACE_DIM= 8 

# Same method as above
def load_fsdd(spectrograms_path):
    dataset = {}
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:

            if True: # re.match(regex, file_name):
                file_path = os.path.join(root, file_name)
                spectrogram = np.load(file_path) # (n_bins, n_frames, 1) 
                dataset[file_name] = spectrogram[..., np.newaxis]
    return dataset
SPECTROGRAMS_PATH = "/content/drive/MyDrive/Music/VAE/spectrogram"

LATENT_REPRESENTATIONS_PATH = '/content/drive/MyDrive/Music/VAE/model-' + str(LATENT_SPACE_DIM) + '-' + str(BATCH_SIZE) + '-' + str(EPOCHS) + '/'
MODEL_PATH = '/content/drive/MyDrive/Music/VAE/model-8-50-100'

def select_random_images(dataset, num_images=10):
    sample_keys = np.random.choice(list(dataset.keys()), num_images)
    sample_ds = { key: dataset[key] for key in sample_keys }

    return sample_ds


def plot_reconstructed_images(images, reconstructed_images):
    num_images = len(images)
    for i, (image, reconstructed_image) in enumerate(zip(images, reconstructed_images)):

        fig, ax = plt.subplots()
        image = image.squeeze()
        img = librosa.display.specshow(image, y_axis='log', x_axis='time', ax=ax)
        fig.colorbar(img, ax=ax, format="%+2.0f dB")

        fig, ax = plt.subplots()
        reconstructed_image = reconstructed_image.squeeze()
        recon_img = librosa.display.specshow(reconstructed_image, y_axis='log', x_axis='time', ax=ax)
        fig.colorbar(recon_img, ax=ax, format="%+2.0f dB")
    plt.show()


def plot_images_encoded_in_latent_space(latent_representations, sample_labels):
    plt.figure(figsize=(10, 10))
    plt.scatter(latent_representations[:, 0],
                latent_representations[:, 1],
                cmap="rainbow",
                c=sample_labels,
                alpha=0.5,
                s=2)
    plt.colorbar()
    plt.show()

def download_vectors(autoencoder, sample_ds, download_path):
    images = np.array(list(sample_ds.values()))
    filenames = sample_ds.keys()
    latent_representations = autoencoder.encoder.predict(images)

    with open(download_path + 'embeddings.tsv', 'w', newline='') as f_output:
        tsv_output = csv.writer(f_output, delimiter='\t')
        tsv_output.writerows(latent_representations)
    # Write corresdonding filenames
    with open(download_path + 'embedding-filenames.tsv', 'a') as f_output:
        f_output.seek(0)
        f_output.truncate()
        for data in filenames:
            f_output.write(data)
            f_output.write('\n')
    return latent_representations, filenames


autoencoder = VAE.load(MODEL_PATH)
dataset = load_fsdd(SPECTROGRAMS_PATH)

# num_sample_images_to_show = 30

# sample_ds = select_random_images(dataset, num_sample_images_to_show)
# reconstructed_images, _ = autoencoder.reconstruct(np.array(list(dataset.values())))
download_vectors(autoencoder, dataset, LATENT_REPRESENTATIONS_PATH)

# plot_reconstructed_images(sample_images, reconstructed_images)

# num_images = 6000
# sample_images, sample_labels = select_images(x_test, y_test, num_images)
# _, latent_representations = autoencoder.reconstruct(sample_images)
# plot_images_encoded_in_latent_space(latent_representations, sample_labels)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


(array([[-1.3507284 ,  4.4822617 , -0.10582623, ...,  6.527222  ,
         -3.1556602 , -5.7128477 ],
        [-4.629962  , -2.6926074 ,  6.3419995 , ...,  8.127062  ,
          3.2758157 , -9.508995  ],
        [-5.260143  ,  2.5312533 ,  5.0127697 , ...,  3.876795  ,
         -4.816131  , -6.3538876 ],
        ...,
        [-7.203212  , -0.48487404, -1.1823151 , ...,  9.98615   ,
         -0.6588042 , -9.264013  ],
        [-7.66831   , -4.2648206 ,  2.534174  , ..., 11.828755  ,
          0.9495035 , -6.0314183 ],
        [-9.068106  ,  0.07657731,  0.88403106, ...,  7.521008  ,
         -3.9623814 , -7.3862967 ]], dtype=float32),
 dict_keys(['00001-04 01A US Double Nrm.wav.npy', '00001-06 01A US Double Nrm.wav.npy', '00001-09 01A US Double Nrm.wav.npy', '00001-05 01A US Double Nrm.wav.npy', '00001-12 01A US Double Nrm.wav.npy', '00001-02 01A US Double Nrm.wav.npy', '00001-07 01A US Double Nrm.wav.npy', '00001-10 01A US Double Nrm.wav.npy', '00001-11 01A US Double Nrm.wav.npy', '000